<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# LAB: Night light
## Samantha Burton

# Introduction

The goal of this lab is to connect and collect data from analog devices and capture them in digital form using an Analog-Digital Converter. 

You will configure an LED to turn on/off based on the amount of ambient light detected with a photoresistor. 

You will also use a potentiometer to set the threshold indicating when the LED needs to be turned on/off. 

<img src="https://www.dropbox.com/s/u628vjn2uc5h3ua/notebook.png?raw=1" width="10%" align="right">

Read through
* the [ADC notebook](h_ADC.ipynb)  to learn how to measure analog voltages;
* the [Potentiometer notebook](h_Potentiometer.ipynb)  to learn how to control voltages;
* the [Photoresistor notebook](h_Photoresistor.ipynb)  to understand how to sense light.

**Build all circuits to ensure that everything works correctly.**

# Pre-lab questions

### Potentiometer
1. How does the output voltage change as you turn the knob of the potentiometer?
    * The output voltage goes up or down depending which way the potentiometer is turned. It goes up when the knob is turned clockwise, and down when turned counterclockwise.
2. What do you think would happen if you connected the output of the potentiometer to a GPIO pin? (Try it!)
    * The GPIO would read the voltage of the potemtiometer. This is how we are able to use the potentiometer in this lab.
3. What could happen if you connected 3.3V and GND to an outer and the middle pins? (Don't try it!)
    * There could be a short if the knob is turned all the way.

### Photoresistor
1. How must the photoresistor be wired in order to use it? (Hint: this configuration has a name!)
    * The wiring configuration of the photoresistor is called a voltage divider. The resistor is connected to the photoresistor at the same intersection as the GPIO. At the other side, the photoresistor is connected to Voltage. Resistor is always connected to Ground.
2. How does the resistance change with light intensity?
    * When light intensity increases, resistance decreases. When light intensity decreases, resistance increases.
3. What range of resistances do you measure from the photoresistor?
    * The photoresistors have ranges from 1 to 10 kOhms.

# Directions

To create a night light, use the photoresistor to detect when it gets dark (you can simulate darkness by covering the sensor with your hand), then turn on an LED. 

You will use the potentiometer to set a threshold for the darkness level at which to turn on the LED.

* Measure voltages from the potentiometer and photoresistor.
* Calculate the photoresistor resistance from the measured voltage.

* Use the potentiometer voltage to determine the **threshold resistance** of the photoresistor.
    * Map linearly the potentiometer voltage to the photoresistor resistance range.
    
* When the photoresistor resistance is above the **threshold resistance**, turn on the LED.
* When the photoresistor resistance is below the **threshold resistance**, turn off the LED. 

## Hints

To compute the photoresistor **threshold resistance** from the potentiometer voltage, think this way:

* The potentiometer voltage is between $V_{min}=0$V and $V_{max}=3.3$V.
* The photoresistor resistance is between $R_{min}=1$k$\Omega$ and $R_{max}=10$k$\Omega$.
* We seek a linear relation between the measured voltage (of the potentiometer) and the target resistance (of the photoresistor).
$$ R = m V + b$$
* Compute the slope $m$ by
$$ m = \dfrac{R_{max}-R_{min}}{V_{max}-V_{min}} $$

* Compute the intercept $b$ by:
$$ b = R_{min} - m V_{min}$$

* Therefore, for a **potentiometer voltage** $V$, we can compute the **threshold resistance** of the photoresistor as
$$ R_{threshold} = m V_{potentiometer} + b$$

To make it easier for us to test your code, connect 
* the photoresistor on the ADC CH0 
* the potentiometer on the ADC CH1 
* the LED to GPIO pin 27.

Specify the connections if you do extra credit with different hardware.

## Extra credit:
Use multiple LEDs to generate more or less artificial light for different conditions of the ambient light. 

# Starter code

Starter code is provided below. Use comments throughout your code so we can understand what it does. Even if you can't figure out how to write the code for something, a comment explaining what you wanted will get partial credit.

In [19]:
import numpy as np

In [20]:
#set up ADC
import adcUtil as adc



In [21]:
#set up LED
import RPi.GPIO as GPIO

GPIO.setmode(GPIO.BCM)
#assign pin
greenPin = 20
bluePin = 21
#make output stream
GPIO.setup(greenPin,GPIO.OUT)
GPIO.setup(bluePin,GPIO.OUT)



In [22]:
#set up photoresistor
photo = adc.readADC(channel=1)
print(photo)

2.551612903225806


In [23]:
#set up potentiometer
pot = adc.readADC(channel=0)
print(pot)

0.0064516129032258064


In [24]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [42]:
GPIO.setmode(GPIO.BCM)
#assign pin
###########################################################This is where the light pins are assigned
greenPin = 20
bluePin = 21
#make output stream
GPIO.setup(greenPin,GPIO.OUT)
GPIO.setup(bluePin,GPIO.OUT)

potMax = 3.3
potMin = 0.0032258064516129032
photoAmb = 2.5483870967741935
photoMin = 1.461290322580645

photoGrad = np.linspace(photoMin,photoAmb)
potGrad = np.linspace(potMin, potMax)

try:
    
    while True: # loop forever
        
        # your code here
        #read inputs from sensors
        ##################################################This is where the sensor ADC channels are assigned
        photo = adc.readADC(channel=1)
        pot = adc.readADC(channel=0)

        # use the linspace gradients to equate potentiometer value to photoresistor value
        thresholdidx = find_nearest(potGrad,value=pot)
        threshold = photoGrad[thresholdidx]

        #if photoresistor value is less than the potentiometer threshold, blue light on. Otherwise off
        if photo < threshold:
            GPIO.output(bluePin, True)
        else:
            GPIO.output(bluePin, False)
            
        #If the photoresistor value is less than half of the threshold value, green light also on. Otherwise off
        if photo < photoGrad[int(thresholdidx/2)]:
            GPIO.output(greenPin, True)
        else:
            GPIO.output(greenPin, False)
        

except(KeyboardInterrupt, SystemExit):
    print("Interrupt!")

finally:
    print("Done!")
    GPIO.cleanup() # close GPIO

Interrupt!
Done!


# Post lab

<img src="http://www.dropbox.com/s/fcucolyuzdjl80k/todo.jpg?raw=1" width="10%" align="right">

Before you submit the lab, make sure everything works as you expect by restarting the kernel: select **Kernel > Restart & Run All**.

Answer the following questions.

1. What elements of this lab did you find easy/challenging?
    * This lab came more easily to me than some of the others. I spent all of my time in class solving an issue unrelated to this lab, but once I actually got to wiring and coding it didn't take too long.
2. What did you like/dislike about this lab?
    * I didn't dislike anything about this lab. Every time I use the LEDs, though, I'm afraid I'll burn out the bulb.
3. If you did anything worthy of extra credit, tell us about it here!
    * I did the extra credit part of this assignment

# Submit
* Make sure to update your name and department in the top markdown cell.

* Rename the Jupyter notebook with the following convention:
**HL?-FirstLast.ipynb** (replace ? with the lab number)

* Turn in your Jupyter notebook on Canvas. Email submissions don't count.

* Include pictures of all the circuits you made.

![image](20230928_174714.jpg)

# Honor code
Unless explicitly specified, labs are **individual exercises**. Your submission is subject to the [**Mines Honor Code**](http://inside.mines.edu/~epoeter/_GW/CSMHonorCodeUndergradHandbook.pdf).